This code shows CNN model with 4 convolutional layer with 2 fully connected layers

In [1]:
# Library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping



In [2]:
# Standard Normal Variate Function
def snv(input_data):
  
    # Define a new array and populate it with the corrected data  
    output_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]): 
        # Apply correction
        output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / np.std(input_data[i,:])
        
    return output_data

In [3]:
# load the dataset (binary data)
data = pd.read_csv('/content/Acetaminophen_mixture_CNN_Binary.csv', index_col=0)
data = pd.DataFrame(data)
data.drop(['label_name'], axis=1, inplace=True)
data.rename(columns={'Unnamed: 0': 'id'},inplace=True)
y = data['label']
X = data.drop(['label'],axis=1)
X = X.values[:,1:]
X = snv(X)
X = pd.DataFrame(X)
print((X.shape))


(900, 226)


In [4]:
# Train Test Split and reshaping the data
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)
X_train = X_train.values.reshape(X_train.shape[0],X_train.shape[1],1).astype('float32')
X_test= X_test.values.reshape(X_test.shape[0],X_test.shape[1],1).astype('float32')

In [5]:
# Define Model

tf.keras.metrics.RootMeanSquaredError(
    name="root_mean_squared_error", dtype=None
)
n_timesteps = X_train.shape[1] 
n_features  = X_train.shape[2]  
model = keras.Sequential(name="model_conv1D")
model.add(keras.layers.Input(shape=(n_timesteps,n_features)))
model.add(keras.layers.Conv1D(filters=64, kernel_size=7, activation='relu', name="Conv1D_1"))
model.add(keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu', name="Conv1D_2"))
model.add(keras.layers.Conv1D(filters=16, kernel_size=2, activation='relu', name="Conv1D_3"))
model.add(keras.layers.Conv1D(filters=8, kernel_size=1, activation='relu', name="Conv1D_4"))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu', name="Dense_1"))
model.add(keras.layers.Dense(n_features, name="Dense_2"))


model.compile(loss ='mse', optimizer='adam',metrics=['mse','mae',tf.keras.metrics.RootMeanSquaredError()])

In [6]:
model.summary()

Model: "model_conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1D_1 (Conv1D)           (None, 220, 64)           512       
                                                                 
 Conv1D_2 (Conv1D)           (None, 216, 32)           10272     
                                                                 
 Conv1D_3 (Conv1D)           (None, 215, 16)           1040      
                                                                 
 Conv1D_4 (Conv1D)           (None, 215, 8)            136       
                                                                 
 flatten (Flatten)           (None, 1720)              0         
                                                                 
 Dense_1 (Dense)             (None, 32)                55072     
                                                                 
 Dense_2 (Dense)             (None, 1)                

In [7]:
history = model.fit(X_train, y_train, epochs=20,verbose=1)
mse = model.evaluate(X_test, y_test)
print('Mean Squared Error!',mse)


Epoch 1/20
23/23 [==============================] - 3s 55ms/step - loss: 0.0989 - mse: 0.0989 - mae: 0.2674 - root_mean_squared_error: 0.3145
Epoch 2/20
23/23 [==============================] - 1s 52ms/step - loss: 0.0216 - mse: 0.0216 - mae: 0.1213 - root_mean_squared_error: 0.1469
Epoch 3/20
23/23 [==============================] - 1s 52ms/step - loss: 0.0130 - mse: 0.0130 - mae: 0.0898 - root_mean_squared_error: 0.1141
Epoch 4/20
23/23 [==============================] - 1s 56ms/step - loss: 0.0080 - mse: 0.0080 - mae: 0.0695 - root_mean_squared_error: 0.0893
Epoch 5/20
23/23 [==============================] - 1s 64ms/step - loss: 0.0069 - mse: 0.0069 - mae: 0.0648 - root_mean_squared_error: 0.0831
Epoch 6/20
23/23 [==============================] - 1s 65ms/step - loss: 0.0064 - mse: 0.0064 - mae: 0.0620 - root_mean_squared_error: 0.0798
Epoch 7/20
23/23 [==============================] - 2s 68ms/step - loss: 0.0055 - mse: 0.0055 - mae: 0.0578 - root_mean_squared_error: 0.0744
Epoch 

In [8]:
#Predicting Mystery Data
myst = pd.read_csv('/content/Mystery_testing_data.csv', index_col=0)
myst_actual = pd.read_csv('/content/Mystery_labels.csv')
myst_actual = myst_actual['Actual ']
myst = pd.DataFrame(myst)
myst_predictions = pd.DataFrame(myst["label_name"])
myst.drop(['label_name'], axis=1, inplace=True)
# Applying SNV to myst data
myst = myst.values[:,1:]
myst = snv(myst)
myst = pd.DataFrame(myst)

myst= myst.values.reshape(myst.shape[0],myst.shape[1],1).astype('float32')
preds = model.predict(myst)

se = pd.Series(preds.tolist())
myst_predictions.insert(loc=0, column='predictions', value=se.values)
#Saving results
myst_predictions.to_csv('/content/Mystery_Prediction_1DCNN_data_binary.csv')
# Evaluate model
mse = model.evaluate(myst, myst_actual)
print('Mean Squared Error!',mse)


4/4 [==============================] - 0s 8ms/step - loss: 0.1271 - mse: 0.1271 - mae: 0.2051 - root_mean_squared_error: 0.3565
Mean Squared Error! [0.12706543505191803, 0.12706543505191803, 0.2050706148147583, 0.35646238923072815]


In [9]:
#Predicting Ternary Samples Data
data = pd.read_csv('/content/ternary_samples.csv', index_col=0)
data = pd.DataFrame(data)
ternary_predictions = pd.DataFrame()

ternary_predictions["label_name"] =data["label_name"]
ternary_predictions['Actual'] = data['label']

data.drop(['label_name'], axis=1, inplace=True)
data.rename(columns={'Unnamed: 0': 'id'},inplace=True)

y = data['label']
X = data.drop(['label'],axis=1)
X = X.values[:,1:]
X = snv(X)
X = pd.DataFrame(X)

ternary= X.values.reshape(X.shape[0],X.shape[1],1).astype('float32')
preds = model.predict(ternary)

se = pd.Series(preds.tolist())
ternary_predictions.insert(loc=0, column='predictions', value=se.values)
#Saving results
ternary_predictions.to_csv('/content/Ternary_Prediction_1DCNN_data_Ternary.csv')

mse = model.evaluate(ternary, y)
print('Mean Squared Error!',mse)

13/13 [==============================] - 0s 10ms/step - loss: 0.0055 - mse: 0.0055 - mae: 0.0601 - root_mean_squared_error: 0.0739
Mean Squared Error! [0.005465748719871044, 0.005465748719871044, 0.060072802007198334, 0.07393070310354233]


In [10]:
#Predicting Tylenol Data
data = pd.read_csv('tylenol_topcare_samples.csv', index_col=0)
data = pd.DataFrame(data)
ternary_predictions = pd.DataFrame()

ternary_predictions["label_name"] =data["label_name"]


data.drop(['label_name'], axis=1, inplace=True)
data.rename(columns={'Unnamed: 0': 'id'},inplace=True)

X = data.values[:,1:]
X = snv(X)
X = pd.DataFrame(X)

ternary= X.values.reshape(X.shape[0],X.shape[1],1).astype('float32')
preds = model.predict(ternary)

se = pd.Series(preds.tolist())
ternary_predictions.insert(loc=0, column='predictions', value=se.values)
#Saving results
ternary_predictions.to_csv('/content/Tylenol_Prediction_1DCNN.csv')